In [1]:
import pandas as pd
from llama_index import Document, ServiceContext, VectorStoreIndex
from llama_index.llms import OpenAI

# Load your CSV file into a pandas DataFrame
df = pd.read_csv('/home/njui/kn_workspace/curriculum_taxonomy_extractor/data/ALL/ALL_MATH_CSV/math_grade_5_v2.csv')

# Initialize the LLM
llm = OpenAI(model="gpt-3.5-turbo")

# Create the ServiceContext
service_context = ServiceContext.from_defaults(llm=llm)

# Initialize the VectorStoreIndex
index = VectorStoreIndex([], service_context=service_context)


/tmp/ipykernel_2217736/92133626.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [10]:
## Load dotenv
from dotenv import load_dotenv
load_dotenv()


True

In [4]:
# df.head(20)

,---NEW_STRAND---,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,Strand,Sub strand,Specific Learning Outcomes,Suggested Learning Experiences,Key Inquiry\nQuestion(s),NaN,NaN,NaN,NaN,NaN
1,1.0\nNUMBERS,1.0 Whole\nNumbers\n(20 Lessons),"By the end of the sub strand, the\nlearner sho...","<li> In pairs,groups or as individuals identif...",1. Where is\nordering of\nnumbers\nused in rea...,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,i) use IT devices for learning\nmore on whole ...,"<li> In pairs, groups or as individuals identi...",NaN,NaN,NaN,NaN,NaN,NaN
3,Core Competences to be developed:\n<li> Critic...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PCIs:\n<li> Safety; as learners handle place v...,NaN,NaN,NaN,Values:\n<li> Respect; as learners work in pai...,NaN,NaN,NaN,NaN,NaN
5,Links to other subjects:\n<li> Languages; as l...,NaN,NaN,NaN,Suggested Community Services Learning Activiti...,NaN,NaN,NaN,NaN,NaN
6,<li> Agriculture; as learners interact with va...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,No.,Indicators,Exceeds Expectations,Meets Expectations,Approaches\nExpectations,Below Expectations,NaN,NaN,NaN,NaN
8,1,Us e place value\nand total value of\ndigits u...,Uses place value and total value\nof digits up...,Uses place value and\ntotal value of digits up...,Inconsistently uses\nplace value and total\nva...,Little evidence in\nusing place value and\ntot...,NaN,NaN,NaN,NaN
9,2,Us e numbers up to\nhundreds of\nthousands in\...,Uses numbers greater than\nhundreds of thousan...,Uses numbers up to\nhundreds of thousands\nin ...,Inconsistently uses\nnumbers up to\nhundreds o...,Little evidence in\nusing numbers up to\nhundr...,NaN,NaN,NaN,NaN


In [ ]:
# Initialize the VectorStoreIndex
index = VectorStoreIndex([], service_context=service_context)

# Assuming the strands are in the first column, adjust if necessary
column_name = df.columns[0]  # Adjust this if your strand data is in a different column

# Concatenate all rows in the column into a single string, separated by a space
# This assumes that each row in the column is part of one continuous strand data set
all_strands_data = ' '.join(df[column_name].dropna().astype(str))

# Split the text into individual strands by the separator
strands = all_strands_data.split('---NEW_STRAND---')

# Iterate over the extracted strands
for strand in strands:
    # Trim whitespace to clean up the strands
    strand = strand.strip()
    
    # Check if the strand is not empty
    if strand:
        print(strand)  # Optional: for verification
        
        # Create a Document for each non-empty strand
        doc = Document(text=strand)
        
        # Insert the Document into the index
        index.insert(doc)


In [ ]:

# Iterate over the DataFrame rows
for _, row in df.iterrows():
    # Split the text into strands by the separator
    strands = row['text'].split('---NEW_STRAND---')
    
    # Iterate over the strands
    for strand in strands:
        # Create a Document for each strand
        doc = Document(text=strand)
        
        # Insert the Document into the index
        index.insert(doc)

# Now you can query the index to retrieve and format the data
query = 'your_query_here'
results = index.query(query)

# Format the results in the structure you want
formatted_results = {
    "strand": results[0]['strand'],
    "sub_strand": results[0]['sub_strand'],
    "specific_learning_outcome": results[0]['specific_learning_outcome'],
    "suggested_learning_experiences": results[0]['suggested_learning_experiences'],
    "key_inquiry_questions": results[0]['key_inquiry_questions'],
    "assessment_rubrics": [
        {
            "indicators": rubric['indicators'],
            "exceeds_expectations": rubric['exceeds_expectations'],
            "meets_expectations": rubric['meets_expectations'],
            "approaches_expectations": rubric['approaches_expectations'],
            "below_expectations": rubric['below_expectations']
        } for rubric in results[0]['assessment_rubrics']
    ]
}

In [ ]:
################## v2

In [8]:
csv_path = '/home/njui/kn_workspace/curriculum_taxonomy_extractor/data/ALL/ALL_MATH_CSV/math_grade_5_v2.csv'

In [9]:
def process_csv_to_documents(csv_path):
    df = pd.read_csv(csv_path)
    
    # Initialize the document list and temporary storage for current strand data
    documents = []
    current_strand_data = {}
    is_collecting_rubrics = False  # Flag to start collecting rubrics
    
    # Iterate through the DataFrame
    for index, row in df.iterrows():
        if row[0] == '---NEW_STRAND---':
            # If current_strand_data is not empty, create a Document and reset
            if current_strand_data:
                documents.append(create_document_from_strand_data(current_strand_data))
                current_strand_data = {}
            is_collecting_rubrics = False  # Reset rubric collection flag for new strand
            continue
        
        # Detect headers to reset collecting rubrics
        if row[0] == 'Strand':
            is_collecting_rubrics = False
        
        # Start collecting rubrics when reaching the indicators section
        if row[0] == 'No.':
            is_collecting_rubrics = True
            current_strand_data['assessment_rubrics'] = []
            continue
        
        # Collect data based on the row's content
        if not is_collecting_rubrics:
            # Collect general information about the strand
            collect_strand_data(row, current_strand_data)
        else:
            # Collect assessment rubric data
            collect_rubric_data(row, current_strand_data)
    
    # Handle the last strand
    if current_strand_data:
        documents.append(create_document_from_strand_data(current_strand_data))
    
    return documents

def collect_strand_data(row, strand_data):
    # Here, you would collect data for each part of the strand based on the row's content
    # This is a simplified version. You need to adjust according to actual column names and structure
    strand_data['strand'] = row[0] if pd.notna(row[0]) else strand_data.get('strand', '')
    strand_data['sub_strand'] = row[1] if pd.notna(row[1]) else strand_data.get('sub_strand', '')
    strand_data['specific_learning_outcome'] = row[2] if pd.notna(row[2]) else strand_data.get('specific_learning_outcome', '')
    # Extend this logic for other fields like suggested_learning_experiences, key_inquiry_questions

def collect_rubric_data(row, strand_data):
    # Append a new rubric to the 'assessment_rubrics' list within the current_strand_data
    if pd.notna(row[1]):  # Assuming the indicators start from the second column
        rubric = {
            "indicators": row[1],
            "exceeds_expectations": row[2],
            "meets_expectations": row[3],
            "approaches_expectations": row[4],
            "below_expectations": row[5]
        }
        strand_data['assessment_rubrics'].append(rubric)

def create_document_from_strand_data(strand_data):
    # Convert the aggregated strand data into a Document format
    # This function needs to handle converting the structured data into text or a suitable format for Document creation
    # For simplicity, this is a placeholder function
    text_representation = f"{strand_data.get('strand', '')}\n{strand_data.get('sub_strand', '')}\n{strand_data.get('specific_learning_outcome', '')}"
    # Extend this logic to include all parts of the strand_data including assessment_rubrics
    return Document(text=text_representation.strip())

# After defining these functions, call process_csv_to_documents with your CSV path
documents = process_csv_to_documents(csv_path)

# Then, insert each document into the Llama Index
for doc in documents:
    index.insert(doc)


/tmp/ipykernel_2217736/287439567.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[0] == '---NEW_STRAND---':
/tmp/ipykernel_2217736/287439567.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[0] == 'Strand':
/tmp/ipykernel_2217736/287439567.py:24: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[0] == 'No.':
/tmp/ipykernel_2217736/287439567.py:46: FutureWarning: Series.__getitem__ treating keys as positions i

In [ ]:

# Now you can query the index to retrieve and format the data
query = 'your_query_here'
results = index.query(query)

# Format the results in the structure you want
formatted_results = {
    "strand": results[0]['strand'],
    "sub_strand": results[0]['sub_strand'],
    "specific_learning_outcome": results[0]['specific_learning_outcome'],
    "suggested_learning_experiences": results[0]['suggested_learning_experiences'],
    "key_inquiry_questions": results[0]['key_inquiry_questions'],
    "assessment_rubrics": [
        {
            "indicators": rubric['indicators'],
            "exceeds_expectations": rubric['exceeds_expectations'],
            "meets_expectations": rubric['meets_expectations'],
            "approaches_expectations": rubric['approaches_expectations'],
            "below_expectations": rubric['below_expectations']
        } for rubric in results[0]['assessment_rubrics']
    ]
}